# Generating a PROBE Subckt to Control the Switch Matrix Registers to Make Connections

In [1]:
import csv
import json
import os

In [2]:
# Change the working directory to the script's directory

working_dir = "/Users/peterkinget/iCloudDrive/Work/MOSBIUS/MOSbiusTools/MOSbiusCADFlow/MOSbiusV2Tools/MOSbiusV2Tools/tmp/"
os.chdir(working_dir)

# Define the file path
file_path = "MOSbiusV2_sw_matrix_register_map_20250503.csv"

# Check if the script is being run from the correct directory
if not os.path.exists(file_path):
    print(f"Error: The script must be run from the directory containing {filename}.")
    sys.exit(1) 

NameError: name 'filename' is not defined

In [ ]:
def generate_circuit_file(circuit_json_path, pin_map_path, register_map_path, template_path, output_path):
    # Load the circuit JSON
    with open(circuit_json_path, 'r') as circuit_file:
        circuit_data = json.load(circuit_file)

    # Load the pin-to-sw_matrix mapping
    with open(pin_map_path, 'r') as pin_map_file:
        pin_to_sw_matrix = json.load(pin_map_file)

    # Load the sw_matrix-to-register mapping
    with open(register_map_path, 'r') as register_map_file:
        sw_matrix_to_register = json.load(register_map_file)

    # Load the SPICE template
    with open(template_path, 'r') as template_file:
        spice_header = template_file.readlines()

    # Open the output file for writing
    with open(output_path, 'w') as output_file:
        # Write the SPICE template header
        output_file.writelines(f"* {circuit_json_path}\n")
        output_file.writelines(spice_header)

        # Track connected probes
        connected_probes = set()

        # Iterate through each connection in the circuit JSON
        for connection, pins in circuit_data.items():
            for pin in pins:
                # Get the sw_matrix pin number
                sw_matrix_pin = pin_to_sw_matrix.get(pin, None)
                if sw_matrix_pin is None:
                    print(f"Warning: Pin '{pin}' not found in pin_name_to_sw_matrix_pin_number.json")
                    continue

                # Get the register
                try:
                    register = sw_matrix_to_register[str(int(sw_matrix_pin))][connection]
                except KeyError as e:
                    print(f"Error: Key {e} not found in the dictionaries.")
                    continue

                # Write the connection to the SPICE file
                output_file.write(f"* Connection: {connection}, Pin: {pin}, sw_matrix_pin: {sw_matrix_pin}, Register: {register}\n")
                output_file.write(f"V{pin}_to_{connection} PROBE<{int(register)}> VDD 0\n")

                # Mark this probe as connected
                connected_probes.add(int(register))
        # print(connected_probes)
        # By default, connect all unused probes to VSS
        for probe in range(1, 1889):  # 1888 probes
            if probe not in connected_probes:
                output_file.write(f"Vprobe_{probe}_to_VSS PROBE<{probe}> VSS 0\n")
            # else:
            #    print(f"Skipping {probe}")

        # Write the SPICE footer
        output_file.write(".ENDS\n")

        return (sw_matrix_to_register)

In [8]:
(sw_matrix_to_register) = generate_circuit_file(
    circuit_json_path="../../examples/RO3_circuit.json",
    pin_map_path="pin_name_to_sw_matrix_pin_number.json",
    register_map_path="switch_matrix_register_map.json",
    template_path="PK_set_SWMATRIX_template.cir",
    output_path="generated_set_SWMATRIX_PROBE.cir"
)


{323, 324, 325, 298, 299, 363, 364, 365, 338, 339, 349}
Skipping 298
Skipping 299
Skipping 323
Skipping 324
Skipping 325
Skipping 338
Skipping 339
Skipping 349
Skipping 363
Skipping 364
Skipping 365
